# Datasets

In this notebook, you will be guided to :
- Create and upload **datasets** (image, rasters, point cloud, mesh, vector and file)
- Create an **annotation**

#### You may execute this notebook one cell after the other with `Shift + Enter`

For more options, explore `Cell` in the top menu bar, or read a [tutorial](https://www.dataquest.io/blog/jupyter-notebook-tutorial/)

In [1]:
import os
import alteia

In [2]:
import getpass
platform_url = 'https://app.alteia.com'
login = input('Enter your email ')
password = getpass.getpass('Enter your password ')

In [3]:
sdk = alteia.SDK(url=platform_url, user=login, password=password)

# Create the raster datasets

## Orthomosaic

In [48]:
ortho_dataset = sdk.datasets.create_raster_dataset(
  name='Orthomosaic',
  project=my_project.id,
  mission=my_mission.id,
  dataset_format='geotiff',
  categories=['orthomosaic'])

sdk.datasets.upload_file(
  dataset=ortho_dataset.id,
  component='raster',
  file_path='Orthomosaic.tif')

## DSM (Digital Surface Model)

In [ ]:
dsm_dataset = sdk.datasets.create_raster_dataset(
  name='DSM',
  project=my_project.id,
  mission=my_mission.id,
  dataset_format='geotiff',
  categories=['dsm'])

sdk.datasets.upload_file(
  dataset=dsm_dataset.id,
  component='raster',
  file_path='DSM.tif')

# Create the point cloud dataset

In [ ]:
pcl_dataset = sdk.datasets.create_pcl_dataset(
  name='PointCloud',
  project=my_project.id,
  mission=my_mission.id,
  dataset_format='las')

sdk.datasets.upload_file(
  dataset=pcl_dataset.id,
  component='pcl',
  file_path='Point Cloud.las')

# Create the mesh dataset

In [ ]:
mesh_dataset = sdk.datasets.create_mesh_dataset(
  name='Mesh',
  project=my_project.id,
  mission=my_mission.id,
  dataset_format='obj',
  texture_count=1)

sdk.datasets.upload_file(
  dataset=mesh_dataset.id,
  component='mesh',
  file_path='Model.obj')

sdk.datasets.upload_file(
  dataset=mesh_dataset.id,
  component='material',
  file_path='Model.mtl')

sdk.datasets.upload_file(
  dataset=mesh_dataset.id,
  component='texture',
  file_path='Model_0.jpg')

# Create the vector dataset

In [ ]:
vector_dataset = sdk.datasets.create_vector_dataset(
    name='ContoursLines',
    project=my_project.id,
    mission=my_mission.id,
    dataset_format='geojson')

sdk.datasets.upload_file(
  dataset=vector_dataset.id,
  component='vector',
  file_path='Contours.geojson')

# Create an annotation

In [ ]:
my_annotation = sdk.annotations.create(
   project=my_project.id,
   mission=my_mission.id,
   geometry={
       "type": "Point",
       "coordinates": [12.397626065125216, -6.001589143208957]
   },
   name='New annotation',
   type='2d',
   icon=sdk.annotations.Icons.CONVEYOR
)